In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [4]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_de.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #2996

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,202916,202916,5512588105,7/19/20,341,PPP,CONNECTIONS COMMUNITY,3821 Lancaster Pike,Wilmington,DE,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665
1,202917,202917,7963857007,4/8/20,341,PPP,"IRON HILL BREWERY, LLC",2502 W. 6TH ST,WILMINGTON,DE,...,Unanswered,NaN,8506701.80,9/7/21,72.0,Accomodation and Food Services,DELAWARE DISTRICT OFFICE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856
2,202918,202918,5522267710,5/1/20,341,PPP,47TH AND 7TH LLC,222 VALLEY RD,WILMINGTON,DE,...,Unanswered,NaN,NaN,NaN,99.0,0,DELAWARE DISTRICT OFFICE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135
3,202919,202919,6561867002,4/7/20,341,PPP,"M. DAVIS & SONS, INC.",19 Germay Dr,WILMINGTON,DE,...,Unanswered,NaN,7988219.79,8/12/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340
4,202920,202920,6162427002,4/6/20,341,PPP,"WORTH & COMPANY, INC.",6263 Kellers Church Road,PIPERSVILLE,DE,...,Unanswered,NaN,7826701.84,6/16/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345


In [5]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [6]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #1

,LoanNumber,full_add,Lat,Long


In [7]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head()

,LoanNumber,full_add,Lat,Long
0,5512588105,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665
1,7963857007,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856
2,5522267710,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135
3,6561867002,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340
4,6162427002,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/DE_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [7]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #7204

<h4>FIPS Script

In [8]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('10')] #DE
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [12]:
geo_id_list =geo_id 

Length Check

In [13]:
print(len(geo_id_list))

2996


Matching FIPS to DataFrame

In [14]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:7204]

In [15]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5512588105,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,[100030119001029]
1,7963857007,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,[100030024003008]
2,5522267710,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,[100030129002003]
3,6561867002,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,[100030129001009]
4,6162427002,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,[]
...,...,...,...,...,...
2991,7925667008,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,[100030138002006]
2992,8190337202,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,[100030105021008]
2993,9093838307,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,[100050504061037]
2994,9156708410,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,[100030125005007]


Remove brackets

In [16]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5512588105,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,100030119001029
1,7963857007,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,100030024003008
2,5522267710,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,100030129002003
3,6561867002,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,100030129001009
4,6162427002,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
...,...,...,...,...,...
2991,7925667008,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,100030138002006
2992,8190337202,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,100030105021008
2993,9093838307,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,100050504061037
2994,9156708410,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,100030125005007


In [17]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #3

,LoanNumber,full_add,Lat,Long,FIPS_z
4,6162427002,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1582,1745487302,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2635,9523837105,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


Parse for Merge

In [18]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,5512588105,100030119001029
1,7963857007,100030024003008
2,5522267710,100030129002003
3,6561867002,100030129001009
4,6162427002,NaN
...,...,...
2991,7925667008,100030138002006
2992,8190337202,100030105021008
2993,9093838307,100050504061037
2994,9156708410,100030125005007


Read in base data

In [19]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_de.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/DE_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 

#fips_check.head() #9179


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,202916,202916,5512588105,7/19/20,341,PPP,CONNECTIONS COMMUNITY,3821 Lancaster Pike,Wilmington,DE,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665
1,202917,202917,7963857007,4/8/20,341,PPP,"IRON HILL BREWERY, LLC",2502 W. 6TH ST,WILMINGTON,DE,...,Unanswered,NaN,8506701.80,9/7/21,72.0,Accomodation and Food Services,DELAWARE DISTRICT OFFICE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856
2,202918,202918,5522267710,5/1/20,341,PPP,47TH AND 7TH LLC,222 VALLEY RD,WILMINGTON,DE,...,Unanswered,NaN,NaN,NaN,99.0,0,DELAWARE DISTRICT OFFICE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135
3,202919,202919,6561867002,4/7/20,341,PPP,"M. DAVIS & SONS, INC.",19 Germay Dr,WILMINGTON,DE,...,Unanswered,NaN,7988219.79,8/12/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340
4,202920,202920,6162427002,4/6/20,341,PPP,"WORTH & COMPANY, INC.",6263 Kellers Church Road,PIPERSVILLE,DE,...,Unanswered,NaN,7826701.84,6/16/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,205907,205907,7925667008,4/8/20,341,PPP,"ABBY MEDICAL, LLC, SERIES 1",1 CENTURIAN DR Suite 106,NEWARK,DE,...,Unanswered,NaN,150858.90,11/19/20,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759
2992,205908,205908,8190337202,4/28/20,341,PPP,E CATCHER INC,824 Philadelphia Pike,Wilmington,DE,...,Unanswered,NaN,151430.14,4/15/21,51.0,Information,DELAWARE DISTRICT OFFICE,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663
2993,205909,205909,9093838307,1/30/21,341,PPS,LAKESHORE MOTOR COMPANY,22586 Sussex Hwy,Seaford,DE,...,Unanswered,NaN,NaN,NaN,45.0,Retail Trade,DELAWARE DISTRICT OFFICE,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801
2994,205910,205910,9156708410,2/16/21,341,PPS,DENTAL DIAMOND LLC,1 Winston Ave,Wilmington,DE,...,Unanswered,NaN,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426


Merge with MainFrame

In [20]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,202916,202916,5512588105,7/19/20,341,PPP,CONNECTIONS COMMUNITY,3821 Lancaster Pike,Wilmington,DE,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,100030119001029
1,202917,202917,7963857007,4/8/20,341,PPP,"IRON HILL BREWERY, LLC",2502 W. 6TH ST,WILMINGTON,DE,...,NaN,8506701.80,9/7/21,72.0,Accomodation and Food Services,DELAWARE DISTRICT OFFICE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,100030024003008
2,202918,202918,5522267710,5/1/20,341,PPP,47TH AND 7TH LLC,222 VALLEY RD,WILMINGTON,DE,...,NaN,NaN,NaN,99.0,0,DELAWARE DISTRICT OFFICE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,100030129002003
3,202919,202919,6561867002,4/7/20,341,PPP,"M. DAVIS & SONS, INC.",19 Germay Dr,WILMINGTON,DE,...,NaN,7988219.79,8/12/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,100030129001009
4,202920,202920,6162427002,4/6/20,341,PPP,"WORTH & COMPANY, INC.",6263 Kellers Church Road,PIPERSVILLE,DE,...,NaN,7826701.84,6/16/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,205907,205907,7925667008,4/8/20,341,PPP,"ABBY MEDICAL, LLC, SERIES 1",1 CENTURIAN DR Suite 106,NEWARK,DE,...,NaN,150858.90,11/19/20,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,100030138002006
2992,205908,205908,8190337202,4/28/20,341,PPP,E CATCHER INC,824 Philadelphia Pike,Wilmington,DE,...,NaN,151430.14,4/15/21,51.0,Information,DELAWARE DISTRICT OFFICE,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,100030105021008
2993,205909,205909,9093838307,1/30/21,341,PPS,LAKESHORE MOTOR COMPANY,22586 Sussex Hwy,Seaford,DE,...,NaN,NaN,NaN,45.0,Retail Trade,DELAWARE DISTRICT OFFICE,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,100050504061037
2994,205910,205910,9156708410,2/16/21,341,PPS,DENTAL DIAMOND LLC,1 Winston Ave,Wilmington,DE,...,NaN,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,100030125005007


<b>Fill FIPS Columns - if FIPS file

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

In [ ]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #16 , notna #9163 / 9179

Convert to File

In [21]:
fips_DE = fips_merge
#fips_WV = fips_merge_drop

In [22]:
fips_DE.to_csv("data/state_data/geo/geo_fips/DE_fips_scraped.csv") 

Review & Compare

In [2]:
geo_DE_fips= pd.read_csv("data/state_data/geo/geo_fips/DE_fips_scraped.csv") 
geo_DE_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,202916,202916,5512588105,7/19/20,341,PPP,CONNECTIONS COMMUNITY,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"3821 Lancaster Pike, Wilmington, DE",39.757096,-75.595665,1.000301e+14
1,1,1,1,202917,202917,7963857007,4/8/20,341,PPP,"IRON HILL BREWERY, LLC",...,N,8506701.80,9/7/21,72.0,Accomodation and Food Services,DELAWARE DISTRICT OFFICE,"2502 W. 6TH ST, WILMINGTON, DE",39.753040,-75.577856,1.000300e+14
2,2,2,2,202918,202918,5522267710,5/1/20,341,PPP,47TH AND 7TH LLC,...,N,NaN,NaN,99.0,0,DELAWARE DISTRICT OFFICE,"222 VALLEY RD, WILMINGTON, DE",39.726703,-75.582135,1.000301e+14
3,3,3,3,202919,202919,6561867002,4/7/20,341,PPP,"M. DAVIS & SONS, INC.",...,N,7988219.79,8/12/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"19 Germay Dr, WILMINGTON, DE",39.728914,-75.578340,1.000301e+14
4,4,4,4,202920,202920,6162427002,4/6/20,341,PPP,"WORTH & COMPANY, INC.",...,N,7826701.84,6/16/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,2991,2991,2991,205907,205907,7925667008,4/8/20,341,PPP,"ABBY MEDICAL, LLC, SERIES 1",...,N,150858.90,11/19/20,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 CENTURIAN DR Suite 106, NEWARK, DE",39.695892,-75.656759,1.000301e+14
2992,2992,2992,2992,205908,205908,8190337202,4/28/20,341,PPP,E CATCHER INC,...,N,151430.14,4/15/21,51.0,Information,DELAWARE DISTRICT OFFICE,"824 Philadelphia Pike, Wilmington, DE",39.772827,-75.495663,1.000301e+14
2993,2993,2993,2993,205909,205909,9093838307,1/30/21,341,PPS,LAKESHORE MOTOR COMPANY,...,N,NaN,NaN,45.0,Retail Trade,DELAWARE DISTRICT OFFICE,"22586 Sussex Hwy, Seaford, DE",38.670838,-75.593801,1.000505e+14
2994,2994,2994,2994,205910,205910,9156708410,2/16/21,341,PPS,DENTAL DIAMOND LLC,...,N,NaN,NaN,62.0,Health Care and Social Assistance,DELAWARE DISTRICT OFFICE,"1 Winston Ave, Wilmington, DE",39.728615,-75.586426,1.000301e+14


In [3]:
##### fill in missing Lat, Long, FIPS
geo_DE_fips.loc[geo_DE_fips["FIPS_z"].isnull()] #3

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
4,4,4,4,202920,202920,6162427002,4/6/20,341,PPP,"WORTH & COMPANY, INC.",...,N,7826701.84,6/16/21,23.0,Construction,DELAWARE DISTRICT OFFICE,"6263 Kellers Church Road, PIPERSVILLE, DE",40.392992,-75.153345,NaN
1582,1582,1582,1582,204498,204498,1745487302,4/28/20,341,PPP,AMERIBULK TRANSPORT LLC.,...,N,277314.78,10/12/21,48.0,Transportation and Warehousing,DELAWARE DISTRICT OFFICE,"6300 PHILADELPHIA PIKE, CLAYMONT, DE",39.812609,-75.432185,NaN
2635,2635,2635,2635,205551,205551,9523837105,4/15/20,341,PPP,"CLAIM TO FAME STABLE, LLC",...,N,172182.21,8/19/21,71.0,"Arts, Entertainment, and Recreation",DELAWARE DISTRICT OFFICE,"30003 Pawley Island Ct, DAGSBORO, DE",33.433224,-79.121431,NaN


In [4]:
geo_DE_fips["BusinessType"] = geo_DE_fips["BusinessType"].replace({'Limited  Liability Company(LLC)': 'Limited Liability Company(LLC)'})
geo_DE_fips.to_csv("data/state_data/geo/geo_fips/DE_fips_scraped.csv")
listbusiness = geo_DE_fips["BusinessType"].value_counts()
listbusiness

Corporation                       1050
Limited Liability Company(LLC)     903
Subchapter S Corporation           689
Non-Profit Organization            187
Sole Proprietorship                 68
Partnership                         42
Professional Association            23
Limited Liability Partnership       22
501(c)3 – Non Profit                 3
Trust                                2
Cooperative                          2
Non-Profit Childcare Center          2
Independent Contractors              1
Self-Employed Individuals            1
Joint Venture                        1
Name: BusinessType, dtype: int64